In [22]:
import pandas as pd
import pandas_read_xml as pdx
import numpy as np

df = pd.read_xml("test.osm")
# nodes = df[:df[df["id"]==2569].index[0]]
# way = df[df[df["id"]==2569].index[0]:]

# nodes = nodes.set_index("id")
# nodes.index = nodes.index.astype(int)

# nodes = nodes.drop(columns=["tag", "nd"])
# way = way.drop(columns=["lat", "lon", "tag", "nd"])
# way = way.set_index("id")
# way.index = way.index.astype(int)

Series([], Name: tag, dtype: float64)

In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse('test.osm')
root = tree.getroot()

# Nodes

In [6]:
#Retrieving nodes
nodes=[]
for child in root:
    if (child.tag=="node"):
        nodes.append({
            "id": int(child.attrib['id']),
            "lat": float(child.attrib['lat']),
            "lon": float(child.attrib['lon']),
            "tags": {tag.attrib["k"]: tag.attrib["v"] for tag in child}
        })
print(nodes[2])

{'id': 224811748, 'lat': 35.003691, 'lon': 135.7523955, 'tags': {'crossing': 'traffic_signals', 'highway': 'crossing', 'tactile_paving': 'yes'}}


In [ ]:
na_list={}
n_nodes = 439986
for col in nodes.columns:
    nan_n = n_nodes - ((nodes[col] == "nan")|(nodes[col] == "n")|(nodes[col] == "na")).sum()
    #if nan_n <= 436352:
    na_list[col] = nan_n
    
del na_list["lat"]
del na_list["lon"]

import matplotlib.pyplot as plt

In [ ]:
def plot_dict(dic):
    # Sort the dictionary items based on values
    sorted_items = sorted(dic.items(), key=lambda x: x[1])
    
    # Extract sorted keys and values
    sorted_keys = [item[0] for item in sorted_items][::-1]
    sorted_values = [item[1] for item in sorted_items][::-1]
    
    plt.figure(figsize=(200,5))
    plt.bar(sorted_keys, sorted_values)
    plt.xlabel('Columns')
    plt.ylabel('Unique IDs')
    plt.title('Unique IDs for Each Column')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
    plt.show()

plot_dict(na_list)
print("number of nodes : ", nodes.shape[0])

number of nodes :  439986


# Ways

In [ ]:
#Retrieving way info
way_features=[]
nodes_in_way={key: {"nodes" : []} for key in way.index.values}

for child in root:
    #print(child.attrib["id"])
    for schild in child:
        if (child.tag=="way")&(schild.tag == "nd"):
            nodes_in_way[int(child.attrib['id'])]["nodes"] = nodes_in_way[int(child.attrib['id'])]["nodes"] + [schild.attrib["ref"]]

        if (child.tag=="way")&(schild.tag == "tag"):
            nodes_in_way[int(child.attrib['id'])][schild.attrib["k"]] = schild.attrib["v"]
            
for key, sub_dic in nodes_in_way.items():
    for features, value in sub_dic.items():
        way.at[key, features] = value

In [ ]:
na_list2={}
n_way = way.shape[0]
for col in way.columns:
    nan_n = n_way - ((way[col] == "nan")|(way[col] == "na")|(way[col] == "n")).sum()
    #if nan_n <= 436352:
    na_list2[col] = nan_n


In [ ]:
plot_dict(na_list2)
print("number of road : ", way.shape[0])

number of road :  121524


# Feature selection

In [ ]:
nodes_sel = nodes[["lat", "lon", "bicycle", "cycleway", "traffic_calming"]]

way_sel = way[["nodes", "cycleway", "bicycle", "cyclestreet", "maxspeed", "surface", "tracktype",
                "smoothness", "traffic_calming", "lanes", "width", "shoulder", "maxheight", "hgv", "motor_vehicle"]]

In [ ]:
way_sel.to_csv("./ways_feature.csv", index=True)
nodes_sel.to_csv("./nodes_feature.csv", index=True)